In [1]:
import sys
import os 
sys.path.append("/home/zichong/virtual_cell_challenge/wgr/WGR") 

"""
incase the above code does not work, you can use the absolute path instead
sys.path.append(r".\")
"""


'\nincase the above code does not work, you can use the absolute path instead\nsys.path.append(r".")\n'

In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau 

In [3]:
from utils.basic_utils import setup_seed
from data.SimulationData import DataGenerator 
from utils.training_utils import train_WGR_fnn
from utils.evaluation_utils import L1L2_MSE_mean_sd_G, MSE_quantile_G_uniY
from models.generator import generator_fnn
from models.discriminator import discriminator_fnn

In [ ]:
import argparse

if 'ipykernel_launcher.py' in sys.argv[0]:  #if not work in jupyter, you can delete this part
    import sys
    sys.argv = [sys.argv[0]] 



parser = argparse.ArgumentParser(description='Implementation of WGR for M1')

parser.add_argument('--Xdim', default=5, type=int, help='dimensionality of X')
parser.add_argument('--Ydim', default=1, type=int, help='dimensionality of Y')
parser.add_argument('--model', default='M1', type=str, help='model')

parser.add_argument('--noise_dim', default=5, type=int, help='dimensionality of noise vector')
parser.add_argument('--noise_dist', default='gaussian', type=str, help='distribution of noise vector')

parser.add_argument('--train', default=5000, type=int, help='size of train dataset')
parser.add_argument('--val', default=1000, type=int, help='size of validation dataset')
parser.add_argument('--test', default=1000, type=int, help='size of test dataset')

parser.add_argument('--train_batch', default=128, type=int, metavar='BS', help='batch size while training')
parser.add_argument('--val_batch', default=100, type=int, metavar='BS', help='batch size while validation')
parser.add_argument('--test_batch', default=100, type=int, metavar='BS', help='batch size while testing')
parser.add_argument('--epochs', default=100, type=int, help='number of epochs to train')
parser.add_argument('--reps', default=100, type=int, help='number of replications')

args = parser.parse_args()

print(args)

Namespace(Xdim=5, Ydim=1, model='M1', noise_dim=5, noise_dist='gaussian', train=5000, val=1000, test=1000, train_batch=128, val_batch=100, test_batch=100, epochs=100, reps=100)


In [5]:
# Set seed 
setup_seed(1234)

In [6]:
# Generate data from M1
data_gen = DataGenerator(args)
DATA = data_gen.generate_data('M1')
train_X, train_Y = DATA['train_X'], DATA['train_Y']
val_X, val_Y = DATA['val_X'], DATA['val_Y']
test_X, test_Y = DATA['test_X'], DATA['test_Y']

In [7]:
# Create TensorDatasets and initialize a DataLoaders
train_dataset = TensorDataset( train_X.float(), train_Y.float() )
loader_train = DataLoader(train_dataset , batch_size=args.train_batch, shuffle=True)

val_dataset = TensorDataset( val_X.float(), val_Y.float() )
loader_val = DataLoader(val_dataset , batch_size=args.val_batch, shuffle=True)

test_dataset = TensorDataset( test_X.float(), test_Y.float() )
loader_test  = DataLoader(test_dataset , batch_size=args.test_batch, shuffle=True)

In [8]:
# Define generator network and discriminator network
G_net = generator_fnn(Xdim=args.Xdim, Ydim=args.Ydim, noise_dim=args.noise_dim, hidden_dims = [64, 32])
D_net = discriminator_fnn(input_dim=args.Xdim+args.Ydim, hidden_dims = [64, 32])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
G_net.to(device)
D_net.to(device)

Sequential(
  (0): Linear(in_features=6, out_features=64, bias=True)
  (1): LeakyReLU(negative_slope=0.01, inplace=True)
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): LeakyReLU(negative_slope=0.01, inplace=True)
  (4): Linear(in_features=32, out_features=1, bias=True)
)

In [9]:
# Initialize RMSprop optimizers
D_solver = optim.RMSprop(D_net.parameters(),lr = 0.001)
G_solver = optim.RMSprop(G_net.parameters(),lr = 0.001)


In [10]:
# Training
trained_G, trained_D = train_WGR_fnn(D=D_net, G=G_net, D_solver=D_solver, G_solver=G_solver, 
                                     loader_train = loader_train, loader_val=loader_val,
                                     noise_dim=args.noise_dim, Xdim=args.Xdim, Ydim=args.Ydim, batch_size=args.train_batch,
                                     save_path='./', device=device, num_epochs=200)

Mean L1 Loss: 3.427226, Mean L2 Loss: 23.734234


/home/zichong/miniconda3/envs/torch/lib/python3.12/site-packages/torch/autograd/graph.py:829: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:179.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 0 - D Loss: 0.8459, G Loss: 1.2085
Epoch 1 - D Loss: 0.1244, G Loss: 0.8511
Epoch 2 - D Loss: 0.0902, G Loss: 0.9278
Epoch 3 - D Loss: 0.0463, G Loss: 1.2549
Epoch 4 - D Loss: 0.0345, G Loss: 1.3455
Epoch 5 - D Loss: 0.0329, G Loss: 1.2731
Epoch 6 - D Loss: 0.0283, G Loss: 1.1749
Epoch 7 - D Loss: 0.0194, G Loss: 1.1224
Epoch 8 - D Loss: 0.0097, G Loss: 1.0897
Epoch 9 - D Loss: 0.0144, G Loss: 1.0039
Epoch 10 - D Loss: -0.0037, G Loss: 0.9916
Epoch 11 - D Loss: -0.0009, G Loss: 0.9552
Epoch 12 - D Loss: 0.0033, G Loss: 0.9232
Epoch 13 - D Loss: 0.0003, G Loss: 0.8426
Epoch 14 - D Loss: 0.0071, G Loss: 0.8498
Epoch 15 - D Loss: 0.0054, G Loss: 0.8801
Epoch 16 - D Loss: 0.0064, G Loss: 0.8987
Epoch 17 - D Loss: 0.0129, G Loss: 0.8659
Epoch 18 - D Loss: 0.0069, G Loss: 0.9387
Epoch 19 - D Loss: 0.0013, G Loss: 0.9049
Epoch 20 - D Loss: 0.0051, G Loss: 1.0355
Epoch 21 - D Loss: 0.0026, G Loss: 0.9628
Epoch 22 - D Loss: 0.0015, G Loss: 1.0376
Epoch 23 - D Loss: -0.0017, G Loss: 1.0247

In [12]:
# Calculate the L1 and L2 error, MSE of conditional mean and conditional standard deviation on the test data  
test_G_mean_sd = L1L2_MSE_mean_sd_G(G = trained_G.to('cpu'),  test_size = args.test, noise_dim=args.noise_dim, Xdim=args.Xdim, 
                                    batch_size=args.test_batch, loader_dataset = loader_test )

Model: M1, Univariate, Ydim: 1, J_t_size: 50
L1 Loss: tensor([1.1977])
L2 Loss: tensor([2.9525])
MSE Mean: tensor([0.2268])
MSE SD: tensor([0.2009])


In [14]:
# Calculate the MSE of conditional quantiles at different levels.
test_G_quantile = MSE_quantile_G_uniY(G = trained_G.to('cpu'), loader_dataset = loader_test , noise_dim=args.noise_dim, Xdim=args.Xdim, 
                                      test_size = args.test,  batch_size=args.test_batch)

Q_5: 0.5895, Q_25: 0.3080, Q_50: 0.2314, Q_75: 0.4557, Q_95: 0.9007
